# Soil Water Balance

The soil water storage in the rootzone is the result of the balance between water inputs (e.g. precipitation, irrigation) and outputs (e.g. evaporation, runoff). Soil water is always moving, which means that the soil water storage is always changing, at least to some small amount, but when we look at soil physical processes over coarser temporal scales we can make meaningful approximations about the state of the soil for agricultural, hydrological, adn ecological applications.

In this notebook we will implement a soil water balance at daily time-step. The balance will be similar to the widely used crop coefficient method proposed by the United Nations Food and Agriculture Organization. The goal is to provide a sound computation, although extension of the model described here for real-world application would need careful evaluation and local calibration.

The soil water storage at the end of the day will be estimated like this:

$$ S_t = S_{t-1} + P_t + Irr_t - ET_t Kc_t - R_t - D_t - I_t$$

$S_t$ is the soil water storage over the rootzone at the end of the day $t$

$S_{t-1}$ is the soil water storage over the rootzone at the beginning of the day $t$

$P$ is total precipitation and $Irr$ is total irrigation during time $t$. The model will be developed for rainfed conditions, so $Irr$ will be assumed zero.

$I$ is canopy and litter interception. The term $P-I$ is often known as effective precipitation. In this model interception will be assumed constant, which means that $P<=I$ will not become part of the soil profile.

$ET$ is the potential evapotrasnpiration defined for a reference crop growing under well-watered conditions. This term accounts for both soil evaporative losses and plant transpiration.

$Kc$ is an empirical crop coefficient to adjust the potential evapotranspiration for the current crop conditions. This crop coefficeint changes as a function of the phenological stages of the crop.

$R$ is surface runoff, which in this case will be modeled using the curve number method

$D$ is drainage below the bottom of the rootzone, which will be modeled using the Wilcox method.


In [1]:
# Import modules
import pandas as pd
import numpy as np
from bokeh.plotting import show, figure, output_notebook
from bokeh.layouts import column,gridplot
output_notebook()


Loading BokehJS ...

## Auxiliary functions

To decompress our main function we will create a set of helper functions dedicated to some of the components of the soil water balance. While this is not necessary, it will make our code more modular and compact. Sometimes seeking modularity can yield an overly complex tree of functions that, while making the code more compact, it can affect the overall readability of our code.

In [2]:
# Define auxiliary functions for individual processes

def compute_runoff(P,CN):
    """Daily runoff based on the U.S. Soil Conservation Service curve number method"""
    S = 25400/CN - 254; # Maximum soil moisture retention after runoff begins.
    Ia = S * 0.05; # Initial abstraction (Ia). Rainfall before runoff starts to occur.
    if P > Ia:
        R = (P - Ia)**2 / (P - Ia + S);
    else:
        R = 0
    return R

def compute_potential_et(T_avg,solar_rad,altitude):
    """Potential evapotranspiration based on the Priestley-Taylor method."""
    atm_pressure = 101.3 * ((293 - 0.0065 * altitude)/293)**5.26 # kPa
    gamma = 0.000665 * atm_pressure # Psychrometric constant Cp/(2.45 * 0.622 = 0.000665
    delta = 4098 * (0.6108 * np.exp(17.27 * T_avg / (T_avg  + 237.3))) / (T_avg  + 237.3)**2 # Slope saturated vapor pressure
    soil_heat_flux = 0;
    alpha = 0.5
    PET = alpha*delta/(delta+gamma)*(solar_rad-soil_heat_flux)
    return PET


## Load weather data

We will use a long time series of weather data from the ACME weather station of the Oklahoma Mesonet.

In [3]:
# Load data
df = pd.read_csv('../datasets/acme_ok_daily.csv')

# Check for missing values
df.isna().sum()

df.head()

,Date,DOY,TMAX,TMIN,RAIN,HMAX,HMIN,ATOT,W2AVG,ETgrass
0,1/1/05 0:00,1,21.161111,14.272222,0.00,97.5,65.97,4.09,5.194592,1.976940
1,1/2/05 0:00,2,21.261111,4.794444,0.00,99.3,77.37,4.11,3.428788,1.302427
2,1/3/05 0:00,3,5.855556,3.477778,2.54,99.8,98.20,2.98,3.249973,0.349413
3,1/4/05 0:00,4,4.644444,0.883333,7.62,99.6,98.50,1.21,3.527137,0.288802
4,1/5/05 0:00,5,0.827778,-9.172222,24.13,99.4,86.80,1.65,NaN,0.367956


In [4]:
# Fill missing values
df['TMAX'].interpolate(method='pchip', inplace=True)
df['TMIN'].interpolate(method='pchip', inplace=True)
df['HMAX'].interpolate(method='pchip', inplace=True)
df['HMIN'].interpolate(method='pchip', inplace=True)
df['ATOT'].interpolate(method='pchip', inplace=True)
df['W2AVG'].interpolate(method='pchip', inplace=True)
df['RAIN'].interpolate(method='zero', inplace=True)

# Check that we replaced allmissing values
df.isna().sum()

Date        0
DOY         0
TMAX        0
TMIN        0
RAIN        0
HMAX        0
HMIN        0
ATOT        0
W2AVG       0
ETgrass    16
dtype: int64

In [5]:
# Convert dates from string format to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y %H:%M')

# Print first and last dates
print('First date in dataset:',df['Date'][0])
print('Last date in dataset:',df['Date'][len(df['Date'])-1])

First date in dataset: 2005-01-01 00:00:00
Last date in dataset: 2017-06-18 00:00:00


## Main model

A script that takes into account the meteorological data, initial conditions, and functions describing the components of the soil water balance to compute the rootzone soil water storage on a daily time steps.

In [6]:
# Define simulation length
start_date = pd.to_datetime('20-oct-2015')
end_date = pd.to_datetime('1-jun-2016')
N = (end_date - start_date).days + 1
idx_sim = (df['Date']>=start_date) & (df['Date']<=end_date) 

# Location-specific parameters
altitude = 300 # m above sea level

# Crop-specific parameters
T_base = 0
I = 5 # Canopy and crop residue interception in mm/day
Kc_min = 0.2
Kc_max = 1.1
Kc_stages = [Kc_min,Kc_min,Kc_max,Kc_max,Kc_min]
TU_stages = [0,500,1000,1500,2000]

# Soil-specific parameters
z = 1000 # Length of the soil profile in mm
S_max = 0.45*z # Saturation
FC = 0.35*z  # Field capacity
PWP = 0.20*z # Permanent Wilting Point
D_max = 25 # Maximum daily drainage rate
CN = 80 # Curve number

# Pre-compute some variables
dates = df.loc[idx_sim,'Date']
P = df.loc[idx_sim,'RAIN'].values
SR = df.loc[idx_sim,'ATOT'].values
TAVG = df.loc[idx_sim,['TMAX','TMIN']].mean(axis=1).values
PET = compute_potential_et(TAVG,SR,altitude)

# Pre-allocate arrays with NaN values
Kc = np.ones(N)*np.nan
ET = np.ones(N)*np.nan
S = np.ones(N)*np.nan
PAW = np.ones(N)*np.nan
TU = np.ones(N)*np.nan

# Define initial conditions
S[0] = FC
Kc[0] = Kc_min
ET[0] = PET[0]*Kc_min
TU[0] = TAVG[0] - T_base


In [7]:
# Model
for t in range(1, N):
    TU[t] = TU[t-1] + np.maximum(TAVG[t]-T_base, 0)
    Peff = np.maximum(P[t]-I, 0)
    R = compute_runoff(Peff, CN)
    D = D_max*(S[t-1]/S_max)**((D_max/S_max - 1)/(-D_max/S_max)) # Exponent
    Kc[t] = np.interp(TU[t], TU_stages, Kc_stages)
    PAW[t] = (S[t-1] - PWP) / (FC - PWP)
    PAW[t] = np.maximum(PAW[t], 0)
    Ks = np.maximum(1 - np.exp(-10*PAW[t]), 0);
    ET[t] = PET[t] * Kc[t] * Ks
    S[t] = S[t-1] + Peff - ET[t] - R - D
    S[t] = np.min([S[t], S_max])

In [8]:
# Plot simulation
f1 = figure()
f1.line(TU, S)
f1.line(TU,FC, line_dash='dashed', line_color='royalblue')
f1.line(TU,PWP, line_dash='dashed', line_color='tomato')

f2 = figure(x_axis_type='datetime')
f2.line(dates, ET, line_color='royalblue', legend_label='Actual ET')
f2.line(dates, PET, line_color='tomato', legend_label='Potential ET')
f2.legend.location='top_left'

f3 = figure(x_axis_type='datetime')
f3.line(dates, Kc, line_color='green', legend_label='Crop Kc')

f4 = figure(x_axis_type='datetime')
f4.line(dates, TAVG, line_color='green', legend_label='Crop Kc')
f4.line(dates, T_base, line_dash='dashed', line_color='tomato', legend_label='Tbase')

grid = gridplot([[f1,f2],[f3,f4]], plot_width=400, plot_height=300)
show(grid)

## Assumptions

- Interception depends on the amount of foliage, foliage architecture (e.g. leaf angle), the amount and type of crop stubble, and the evaporative demand during the rainfall event. Using a constant value for canopy and stubble interception is an acceptable approximation that filters small precipitation events that will not reach the soil surface, but this value needs to be carefully selected for each simulation scenario.

- Thermal units were computed before running the model. While this assumption will capture the main crop dynamics, more accurate models should adapt the development rate according to the state of the crop in response to past and current environmental conditions.

- The use of crop coefficients is a simple approximation to weight the potential ET of a reference crop to the crop under simulation. The timing of the crop coefficeints in this model is a simple approximation to capture the timing of the crop water demand, but it is certainly an oversimplification.